<a href="https://colab.research.google.com/github/8lu3/dw_matrix_road_sign/blob/master/day_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color,exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'],train['labels']
X_test, y_test = test['features'],test['labels']

if y_train.ndim == 1: y_train=to_categorical(y_train)
if y_test.ndim == 1: y_test=to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model,X_train,y_train,params_fit={}):
  model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

  logdir = os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size',128),
      epochs=params_fit.get('epochs',5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data',(X_train,y_train)),
      callbacks=[tensorboard_callback]
      )
  
  return model

def predict(model_trained,X_test,y_test,scoring=accuracy_score):
  y_test_norm = np.argmax(y_test,axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob,axis=1)
  
  return scoring(y_test_norm,y_pred)

def train_and_predict(model):
  model_trained = train_model(model,X_train,y_train)
  return predict(model_trained,X_test,y_test)

In [0]:
def get_cnn_v5(input_shape,num_classes):
  return Sequential([
    Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=input_shape),
    Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
    Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
    Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024,activation='relu'),
    Dropout(0.3),

    Dense(1024,activation='relu'),
    Dropout(0.3),

    Dense(num_classes,activation='softmax'),
  ])

In [7]:
train_and_predict( get_cnn_v5(input_shape,num_classes) )

Epoch 1/5
272/272 [==============================] - 4s 17ms/step - loss: 2.4784 - accuracy: 0.3081 - val_loss: 1.0290 - val_accuracy: 0.6720
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.8864 - accuracy: 0.7150 - val_loss: 0.2148 - val_accuracy: 0.9373
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.4178 - accuracy: 0.8714 - val_loss: 0.0948 - val_accuracy: 0.9721
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2597 - accuracy: 0.9206 - val_loss: 0.0484 - val_accuracy: 0.9858
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1937 - accuracy: 0.9418 - val_loss: 0.0462 - val_accuracy: 0.9872


0.9684807256235828

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=input_shape),
    Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
    Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'),
    Conv2D(filters=128,kernel_size=(3,3),activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024,activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024,activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes,activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size',128)),
      epochs=5,
      verbose=0
      )
  score = model.evaluate(X_test,y_test,verbose=0)
  accuracy=score[1]
  print(params, 'accuracy{}'.format(accuracy))
  
  return {'loss':-accuracy,'status':STATUS_OK,'model':model}

In [23]:
space = {
    'batch_size': hp.quniform('batch_size',100,200,10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one',0.3,0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',0.3,0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',0.3,0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one',0.3,0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two',0.3,0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.4235116177312659, 'dropout_cnn_block_three': 0.45299414493852985, 'dropout_cnn_block_two': 0.4828248437411796, 'dropout_dense_block_one': 0.49502328227134296, 'dropout_dense_block_two': 0.34924411958977086}
accuracy0.9061224460601807
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.47322722359148317, 'dropout_cnn_block_three': 0.31259764842508636, 'dropout_cnn_block_two': 0.4779417071751756, 'dropout_dense_block_one': 0.41174949215524675, 'dropout_dense_block_two': 0.45977421181605294}
accuracy0.9442176818847656
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.3912619480727385, 'dropout_cnn_block_three': 0.3585759520800883, 'dropout_cnn_block_two': 0.44073742610771516, 'dropout_dense_block_one': 0.5085249993896763, 'dropout_dense_block_two': 0.5465810662433893}
accuracy0.9505668878555298
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.47810663339602155, 'dropout_cnn_block_three': 0.3107843307374302, 'dropout_cnn_block_two': 0.3048307583